In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSNst%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFDSNst%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FDSNst%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSNst%i.log' % k, mode='a')

tf = 0.5*dt

In [8]:
# back up solutions
solverF.backup_current_solutions()
solverS.backup()
first_pc_counts = 0
while True:
    first_pc_counts = first_pc_counts + 1
    tempF.backup()
    # first advance the solid by half time
    solverF.advance(t=tf, dt=0.5*dt)
    # get the half time step flux
    fluxF.curr[:] = solverF.get_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis
    )
    # mapping
    fluxS.curr[:] = fluxF.curr[f2s]
    # update solid flux interface
    solverS.update_flux_interface(interface_tag, fluxS.curr)
    solverS.advance(t=tf, dt=0.5*dt)
    tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
    tempF.curr[:] = tempS.curr[s2f]
    tempF.update_res()
    is_conv = conv_mntr.determine_convergence(tempF)
    if is_conv or first_pc_counts >= max_pc_steps:
        solverF.put_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis,
            data=tempF.curr
        )
        break
    else:
        under_relax.determine_omega(tempF)
        under_relax.update_solution(tempF)
        solverF.put_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis,
            data=tempF.curr
        )
        solverF.restore_previous_solutions()
        solverS.restore()
    
print(first_pc_counts)
print(tempF.curr)
temps_prev = tempS.curr.copy()
fluxf_prev = fluxF.curr.copy()

6
[307.70494426 307.92026772 308.03798866 308.11157441 308.16174248
 308.19698553 308.2235328  308.24475487 308.25889167 308.26913834
 308.28120164 308.2875629  308.29329218 308.29831807 308.30263745
 308.30646791 308.30927623 308.30993101 308.31190818 308.3134869
 308.31489748 308.3163412  308.31681487 308.31893488 308.31667546
 308.31953686 308.31962534 308.32127512 308.31997986 308.32002311
 308.32145784 308.32003192 308.32104934 308.32177112 308.32093163
 308.3223663  308.31950323 308.32186895 308.32146516 308.32254328
 308.3228016  308.32124166 308.32161587 308.32175145 308.32179001
 308.32210895 308.32204548 308.32232954 308.32240368 308.3199953
 308.32295938 308.32157262 308.32185074 308.32155809 308.32090035
 308.32278883 308.32126767 308.32192899 308.3219158  308.32219411
 308.32222804 308.31977974 308.3226141  308.32118464 308.32204293
 308.32135042 308.32037215 308.32184892 308.32047795 308.32051333
 308.32190412 308.32036602 308.32043226 308.3177428  308.3202373
 308.318371

In [9]:
# begin to couple
for step in range(N):
    t += dt
    tf += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # we need to use physically stable previous temperature and flux
    temps_prev[:] = tempS.curr
    fluxf_prev[:] = fluxF.curr
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        fluxS.backup()
        tempS.backup()
        fluxF.backup()
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = 0.5*(tempS.curr[s2f]+temps_prev[s2f])
        
        solverF.put_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis,
            data=tempF.curr
        )
        
        # advance fluid
        solverF.advance(t=tf, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        fluxS.curr[:] = 0.5*(fluxF.curr[f2s]+fluxf_prev[f2s])
        
        # update residual
        fluxS.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxS)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.update_flux_interface(interface_tag, fluxS.curr)
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(fluxS)
            under_relax.update_solution(fluxS)
            solverS.update_flux_interface(interface_tag, fluxS.curr)
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=5.
step=1, pc_iterations=6.
step=2, pc_iterations=5.
step=3, pc_iterations=5.
step=4, pc_iterations=5.
step=5, pc_iterations=5.
step=6, pc_iterations=5.
step=7, pc_iterations=5.
step=8, pc_iterations=5.
step=9, pc_iterations=5.
step=10, pc_iterations=5.
step=11, pc_iterations=5.
step=12, pc_iterations=5.
step=13, pc_iterations=5.
step=14, pc_iterations=5.
step=15, pc_iterations=5.
step=16, pc_iterations=5.
step=17, pc_iterations=5.
step=18, pc_iterations=5.
step=19, pc_iterations=5.
step=20, pc_iterations=5.
step=21, pc_iterations=5.
step=22, pc_iterations=5.
step=23, pc_iterations=5.
step=24, pc_iterations=5.
step=25, pc_iterations=5.
step=26, pc_iterations=4.
step=27, pc_iterations=4.
step=28, pc_iterations=4.
step=29, pc_iterations=4.
step=30, pc_iterations=4.
step=31, pc_iterations=4.
step=32, pc_iterations=4.
step=33, pc_iterations=4.
step=34, pc_iterations=4.
step=35, pc_iterations=4.
step=36, pc_iterations=4.
step=37, pc_iterations=4.
step=38, pc_iterations

In [10]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFDSNst%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.88448124 0.89278421 0.89866458 0.90340454 0.90737194 0.91067982
 0.91357829 0.91612919 0.91842573 0.92055093 0.92246063 0.9242328
 0.92585995 0.92737601 0.92877652 0.93008483 0.93130149 0.93244637
 0.93351296 0.93450866 0.9354424  0.93631554 0.93713731 0.9379041
 0.93862196 0.939293   0.93992535 0.94051537 0.94106981 0.94158791
 0.94207097 0.9425242  0.94294623 0.94333978 0.9437091  0.94405081
 0.94436893 0.94466322 0.94493794 0.94519085 0.94542452 0.94564054
 0.94583759 0.9460178  0.94618176 0.9463299  0.94646316 0.94658167
 0.94668593 0.94677744 0.94685458 0.94691934 0.94697071 0.94700953
 0.94703629 0.94704973 0.94705211 0.94704111 0.94701805 0.94698215
 0.94693339 0.94687256 0.94679742 0.94670917 0.94660545 0.94648801
 0.94635557 0.94620605 0.94604095 0.94585749 0.94565509 0.94543424
 0.94519119 0.94492799 0.94463998 0.94432739 0.9439853  0.9436164
 0.94321624 0.9427803  0.94230986 0.94179625 0.94123937 0.94063794
 0.93998014 0.9392652  0.93848164 0.93762775 0.93668893 0.9356508